### Import packages

In [76]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import soundfile as sf
import yaml
import math
import datetime
import time

### Choose file to inspect

In [77]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('timestamp_csv/*.csv')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='CSV files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='CSV files:', options=('timestamp_csv/time_stamps_2020-12-15--23-21-29--00-40-00--A.csv',…

timestamp_csv/time_stamps_2017-02-04--10-14-06--00-16-35--BC_2.csv


### Read csv file as pandas

In [78]:
filename = directory + file[0]
time_stamps=pd.read_csv(filename, index_col=0)

# Also extract recording name for later use
recording_name=file[0][26:-4]

In [79]:
recording_name

'2017-02-04--10-14-06--00-16-35--BC_2'

### Read metadata

In [80]:
# Set sample rate
try:
    wave_file=wave.open(file[0], "rb")
    sample_rate = wave_file.getframerate()
except:
    sample_rate=int(input('Please check sampling rate manually in the metadata file and set below in Hz! \n'))
    
# Set raw unit of signal
raw_signal_unit=input('Please check the raw unit of the signal and enter below \n ')
    
# Set reference value
ref_value=int(input('Please check reference value in the metadata files and set below!  \n'))

# Calibration value
cal_value=int(input('If available, please check calibration value of hydrophons in the metadata files and set below! \n what does the fullscale voltage correspond to in Pa? \n '))


# Set researcher
researcher= input('Please indicate the researcher who performs the crop by initials [SG, JR] \n ')

Please check sampling rate manually in the metadata file and set below in Hz! 
100000
Please check the raw unit of the signal and enter below 
 V
Please check reference value in the metadata files and set below!  
1
If available, please check calibration value of hydrophons in the metadata files and set below! 
 what does the fullscale voltage correspond to in Pa? 
 1
Please indicate the researcher who performs the crop by initials [SG, JR] 
 SG


### Read in wav

In [81]:
recording_name=recording_name[:-2]

y, sr=librosa.load('wav_files/' +recording_name + '.wav', sr=sample_rate)

### Start time in seconds since epoch

In [82]:
start_datetime_str=recording_name[:20]
start_datetime_abs=datetime.datetime.strptime(start_datetime_str, '%Y-%m-%d--%H-%M-%S').timestamp()

### Onsets & offsets in csv are in seconds => Convert to samples by multiplying them with the fs

second * fs = samples


In [83]:
time_stamps.onset=time_stamps.onset.apply(lambda x: math.floor(x)) # Round down
time_stamps.offset=time_stamps.offset.apply(lambda x: math.ceil(x)) # Round up 

time_stamps['onset_samples']=time_stamps.onset.apply(lambda x: x*sr) 
time_stamps['offset_samples']=time_stamps.offset.apply(lambda x: x*sr) 

### Function for cropping wav and saving

In [84]:
try:
    os.mkdir('./call_segments/' + recording_name)
    
except Exception as e:
    print(e)

[Errno 17] File exists: './call_segments/2017-02-04--10-14-06--00-16-35--BC'


In [156]:
def segment_onset_utc(row, start_datetime_abs):
    # segment_onset_utc
    onset_segment_utc=start_datetime_abs+row.onset
    onset_utc_str=datetime.datetime.fromtimestamp(onset_segment_utc).strftime("%y-%m-%d--%H-%M-%S")
    
    return onset_utc_str
    
def segment_duration(row):
    #duration_seconds
    duration_segment=row.offset-row.onset
    duration_utc_str=time.strftime('--%H-%M-%S', time.gmtime(duration_segment))
    
    return duration_utc_str
    
        
def recording_file_name(row, recording_name):
    
    call_filename= row.onset_utc_str + row.duration_utc_str
    
    #Extract and add if recorder info exists
    try:
        recorder=recording_name[30:]
        call_filename=call_filename+recorder + '.wav'
    except:
        print('recorder not specified')
        call_filename=call_filename  + '.wav'
        
    return call_filename


def crop_call(row, y, sr, recording_name, start_datetime_abs):
    call=y[row.onset_samples: row.offset_samples]  # Interval closed at the beginning and open at the end
    call_path='./call_segments/' + recording_name +'/' +  row.recording_file_name 
    sf.write(call_path, call, sr) 
    
    return call_path

### Apply functions

In [157]:
time_stamps['onset_utc_str']=time_stamps.apply(lambda x: segment_onset_utc(x, start_datetime_abs), axis=1)
time_stamps['duration_utc_str']=time_stamps.apply(lambda x: segment_duration(x), axis=1)
time_stamps['recording_file_name']=time_stamps.apply(lambda x: recording_file_name(x, recording_name), axis=1)
time_stamps['call_wav_path']= time_stamps.apply(lambda x: crop_call(x, y, sr, recording_name, start_datetime_abs), axis=1)

### Prepare a metadata dataframe

In [158]:
def prep_metadata(time_stamps, sample_rate, ref_value, cal_value, researcher, raw_signal_unit, recording_name):
    meta_df=pd.DataFrame()
    
    meta_df['segment_file']=time_stamps.apply(lambda x: x.recording_file_name, axis=1)
    meta_df['wav_origin']=pd.Series([recording_name]*len(time_stamps))
    
    meta_df['sample_rate']=pd.Series([sample_rate]*len(time_stamps))
    meta_df['raw_signal_unit']=pd.Series([raw_signal_unit]*len(time_stamps))
    meta_df['reference_value']=pd.Series([ref_value]*len(time_stamps))
    meta_df['calibration_value']=pd.Series([cal_value]*len(time_stamps))
    meta_df['researcher']=pd.Series(['SG']*len(time_stamps))
    
    meta_df['onset_samples']=time_stamps.onset_samples
    meta_df['offset_samples']=time_stamps.offset_samples
    meta_df['number_of_samples']=time_stamps.offset_samples-time_stamps.onset_samples
    
    meta_df['onset_seconds']=time_stamps.onset
    meta_df['offset_seconds']=time_stamps.offset
    meta_df['duration_seconds']=meta_df.number_of_samples/meta_df.sample_rate
    
    meta_df['onset_absolute_utc']=time_stamps.onset_utc_str
    
    return meta_df

In [159]:
researcher='SG'
meta_df=prep_metadata(time_stamps, sample_rate, ref_value, cal_value, researcher, raw_signal_unit, recording_name)

In [160]:
meta_df

,segment_file,wav_origin,sample_rate,raw_signal_unit,reference_value,calibration_value,researcher,onset_samples,offset_samples,number_of_samples,onset_seconds,offset_seconds,duration_seconds,onset_absolute_utc
0,17-02-04--10-28-08--00-00-03--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,84200000,84500000,300000,842,845,3.0,17-02-04--10-28-08
1,17-02-04--10-28-15--00-00-03--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,84900000,85200000,300000,849,852,3.0,17-02-04--10-28-15
2,17-02-04--10-28-22--00-00-03--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,85600000,85900000,300000,856,859,3.0,17-02-04--10-28-22
3,17-02-04--10-28-36--00-00-03--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,87000000,87300000,300000,870,873,3.0,17-02-04--10-28-36
4,17-02-04--10-28-45--00-00-03--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,87900000,88200000,300000,879,882,3.0,17-02-04--10-28-45
5,17-02-04--10-29-32--00-00-04--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,92600000,93000000,400000,926,930,4.0,17-02-04--10-29-32
6,17-02-04--10-29-58--00-00-04--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,95200000,95600000,400000,952,956,4.0,17-02-04--10-29-58
7,17-02-04--10-30-17--00-00-04--BC.wav,2017-02-04--10-14-06--00-16-35--BC,100000,V,1,1,SG,97100000,97500000,400000,971,975,4.0,17-02-04--10-30-17


In [164]:
meta_df.segment_file[0][:-3]

'17-02-04--10-28-08--00-00-03--BC.'

### Create metadata files

In [171]:
def write_metadata(row, output_dir):
    
    #Prepare filename
    metafilename=row.wav_origin + '/' +row.segment_file[:-3]  + 'yml'
    
    #Convert row to dictionary
    d = {k: (v if type(v).__module__ != 'numpy' else v.item()) for k,v in row.to_dict().items()}
    
    
    with open(output_dir + metafilename, 'w') as outfile:
        yaml.dump(d, outfile)

In [172]:
output_dir='./call_segments/' 
meta_df.apply(lambda x: write_metadata(x, output_dir), axis =1)

0    None
1    None
2    None
3    None
4    None
5    None
6    None
7    None
dtype: object

### Save csv under new name

In [173]:
time_stamps.to_csv('call_data/call_data_' + recording_name + '.csv')

### Read in metadata

In [ ]:
metafile=''

with open(metafile) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    metadata = yaml.load(file, Loader=yaml.FullLoader)
    
print(metadata)